# Data Preprocessing

Process dataset by topics such as `pull request`, `issue`, `discussion`, and so on.

**Table of Content**
- [Proprocessing Original DataFrame](#preprocessing-original-dataframe)
  - [Convert JSON file to DataFrame](#convert-json-file-to-dataframe)
  - [Extract ChatGPTSharing Table](#extract-chatgptsharing-table)
  - [Extract Mention table](#extract-mention-table)
  - [Extract Conversation Table](#extract-conversation-table)
  - [Clean up](#clean-up)
  - [Save to file](#save-to-file)
  - [Combine to a big dataframe](#combine-to-a-big-dataframe)
- [Merge Different Topics/JSON Files](#merge-different-topicsjson-files)
  - [Read CSV files](#read-csv-files)
  - [Remove Empty Answer and Prompt](#remove-empty-answer-and-prompt)
  - [Filter out Python and English Prompt and Answer](#filter-out-python-and-english-prompt-and-answer)
  - [Unnest Prompt/Answer and Concatenation.](#unnest-promptanswer-and-concatenation)
  - [Save Merged Dataset to File](#save-merged-dataset-to-file)

> NOTE
> 
> 1. All `dataframe.to_csv()` codes are blocked. Feel free to unblock them when saving. 
> 2. All file paths to CSV files are replaced with relative paths.

In [21]:
# Import libraries

import pandas as pd
import helper
import os
import pandas as pd
from langdetect import detect

## Preprocessing Original DataFrame

Filter data by `snapshot_20230727`

### Convert JSON file to DataFrame

The following code iterates the local data directory to read the 'target' JSON file.

To process a specific JSON file, change variable `target` to one of "hn", "pr", "issue", "discussion", "commit", "file" and run through: [Preprocessing Original DataFrame](#preprocessing-original-dataframe)

In [3]:
# Set up file paths and target tables.
snapshots = ["snapshot_20230727"]
dir_path = "~/data/DevGPT/"

"""Change `target` to process other topics.

Options are "hn", "pr", "issue", "discussion", "commit", "file"
"""
# target = "TARGET"
target = "pr"
URL_rename = "URL_" + target

if target == "hn":
    pk = [URL_rename]
else:
    pk = [URL_rename, "RepoName"]

In [5]:
# Read all json file paths.
file_paths = []
for snapshot in snapshots:
    file_path = os.path.join(dir_path, snapshot)
    file_paths += helper.read_filepaths(file_path, target)

In [7]:
# Load Json files.
df = helper.load_dataframes(file_paths)

# Rename URL to URL_[target].
df.rename(columns={"URL": URL_rename}, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147 entries, 0 to 146
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Type               147 non-null    object
 1   URL_pr             147 non-null    object
 2   Author             147 non-null    object
 3   RepoName           147 non-null    object
 4   RepoLanguage       144 non-null    object
 5   Number             147 non-null    int64 
 6   Title              147 non-null    object
 7   Body               147 non-null    object
 8   CreatedAt          147 non-null    object
 9   ClosedAt           130 non-null    object
 10  MergedAt           110 non-null    object
 11  UpdatedAt          147 non-null    object
 12  State              147 non-null    object
 13  Additions          147 non-null    int64 
 14  Deletions          147 non-null    int64 
 15  ChangedFiles       147 non-null    int64 
 16  CommitsTotalCount  147 non-null    int64 
 1

### Extract ChatGPTSharing Table

In [9]:
records = []

pk += ["ChatgptSharing"]

for idx, row in df[pk].iterrows():
    for item in row["ChatgptSharing"]:
        obs = {}
        obs[URL_rename] = row[URL_rename]

        if target != "hn":
            obs["RepoName"] = row["RepoName"]
            
        obs.update(item)
        records.append(obs)

df_chatgpt_sharing = pd.DataFrame(records)

# Rename URL to URL_chatgptsharing.
df_chatgpt_sharing.rename(columns={"URL": "URL_chatgptsharing"}, inplace=True)
df_chatgpt_sharing.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173 entries, 0 to 172
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   URL_pr              173 non-null    object 
 1   RepoName            173 non-null    object 
 2   URL_chatgptsharing  173 non-null    object 
 3   Mention             173 non-null    object 
 4   Status              173 non-null    int64  
 5   DateOfConversation  160 non-null    object 
 6   DateOfAccess        160 non-null    object 
 7   Title               160 non-null    object 
 8   NumberOfPrompts     160 non-null    float64
 9   TokensOfPrompts     160 non-null    float64
 10  TokensOfAnswers     160 non-null    float64
 11  Model               160 non-null    object 
 12  Conversations       160 non-null    object 
 13  HTMLContent         160 non-null    object 
dtypes: float64(3), int64(1), object(10)
memory usage: 19.1+ KB


### Extract Mention table

`MentionedURL` is identical with `URL_[target]`

In [11]:
records = []

pk.pop() 
pk += ["URL_chatgptsharing", "Mention"]

for idx, row in df_chatgpt_sharing[pk].iterrows():

    if not isinstance(row["Mention"], dict):
        continue

    obs = {}
    obs[URL_rename] = row[URL_rename]

    if target != "hn":
        obs["RepoName"] = row["RepoName"]
        
    obs["URL_chatgptsharing"] = row["URL_chatgptsharing"]
    obs.update(row["Mention"])
    records.append(obs)

df_mention = pd.DataFrame(records)
df_mention.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173 entries, 0 to 172
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   URL_pr              173 non-null    object
 1   RepoName            173 non-null    object
 2   URL_chatgptsharing  173 non-null    object
 3   MentionedURL        173 non-null    object
 4   MentionedProperty   173 non-null    object
 5   MentionedAuthor     173 non-null    object
 6   MentionedText       173 non-null    object
 7   MentionedPath       54 non-null     object
dtypes: object(8)
memory usage: 10.9+ KB


### Extract Conversation Table

In [13]:
records = []

pk += ["Conversations"]
for idx, row in df_chatgpt_sharing[pk].iterrows():

    if not isinstance(row["Conversations"], list):
        continue

    for item in row["Conversations"]:
        obs = {}
        obs[URL_rename] = row[URL_rename]

        if target != "hn":
            obs["RepoName"] = row["RepoName"]
            
        obs["URL_chatgptsharing"] = row["URL_chatgptsharing"]
        obs.update(item)
        records.append(obs)

df_conversation = pd.DataFrame(records)
df_conversation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 668 entries, 0 to 667
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   URL_pr              668 non-null    object
 1   RepoName            668 non-null    object
 2   URL_chatgptsharing  668 non-null    object
 3   Prompt              668 non-null    object
 4   Answer              668 non-null    object
 5   ListOfCode          668 non-null    object
dtypes: object(6)
memory usage: 31.4+ KB


### Clean Up
4 tables: `df_[target]`, `df_chatgpt_sharing`, `df_mention`, `df_conversation`

In [15]:
# Remove redundant columns
df = df.drop(columns="ChatgptSharing")
df_chatgpt_sharing = df_chatgpt_sharing.drop(columns=["Mention", "Conversations"])

### Save to file

> NOTE
>
> The default output path is the same as the input. Our outputs will be saved into
> a new folder `cleaned`.
>
> Feel free to unblock the code to save a copy into your local.

In [23]:
# # df dataframe contains the initial dataset
# df.to_csv(
#     os.path.join(dir_path, "cleaned", target + ".csv")
#     )

# # df_chatgpt_sharing dataframe contains chatgptsharing content in target table.
# # It can combine with other tables using URL_'target' and RepoName.
# df_chatgpt_sharing.to_csv(
#     os.path.join(dir_path, "cleaned", target + "_chatgpt_sharing.csv")
#     )

# # df_mention dataframe contains mention content in target table.
# # It can combine with other tables using URL_'target' and RepoName.
# df_mention.to_csv(
#     os.path.join(dir_path, "cleaned", target + "_mention.csv")
#     )

# # df_conversation dataframe contains conversation content in target table.
# # It can combine with other tables using URL_'target' and RepoName.
# df_conversation.to_csv(
#     os.path.join(dir_path, "cleaned", target + "_conversation.csv")
#     )

### Combine to a big dataframe

`df_total` is the dataframe that contains all the information of the target table.
Combining `chatgptsharing`, `mention`, and `conversation`

> NOTE
>
> Combine data from multiple dates can be nasty, the dataframe will become too large to process or save.

In [21]:
# Merge ChatGptSharing table to target table.
if target != "hn":
    merge_on = [URL_rename, "RepoName"]
else:
    merge_on = [URL_rename]

df_total = pd.merge(
    df, df_chatgpt_sharing,
    left_on=merge_on, 
    right_on=merge_on, 
    how="left"
)

In [23]:
# Merge Mention table to target table.
df_total = pd.merge(
    df_total, df_mention,
    left_on=merge_on, 
    right_on=merge_on, 
    how="left"
)

In [25]:
# Merge Conversation table to target table.
df_total = pd.merge(
    df_total, df_conversation,
    left_on=merge_on, 
    right_on=merge_on, 
    how="left"
)

In [27]:
df_total.columns

Index(['Type', 'URL_pr', 'Author', 'RepoName', 'RepoLanguage', 'Number',
       'Title_x', 'Body', 'CreatedAt', 'ClosedAt', 'MergedAt', 'UpdatedAt',
       'State', 'Additions', 'Deletions', 'ChangedFiles', 'CommitsTotalCount',
       'CommitShas', 'CommitSha', 'source_date', 'URL_chatgptsharing_x',
       'Status', 'DateOfConversation', 'DateOfAccess', 'Title_y',
       'NumberOfPrompts', 'TokensOfPrompts', 'TokensOfAnswers', 'Model',
       'HTMLContent', 'URL_chatgptsharing_y', 'MentionedURL',
       'MentionedProperty', 'MentionedAuthor', 'MentionedText',
       'MentionedPath', 'URL_chatgptsharing', 'Prompt', 'Answer',
       'ListOfCode'],
      dtype='object')

In [29]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1784 entries, 0 to 1783
Data columns (total 40 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Type                  1784 non-null   object 
 1   URL_pr                1784 non-null   object 
 2   Author                1784 non-null   object 
 3   RepoName              1784 non-null   object 
 4   RepoLanguage          1778 non-null   object 
 5   Number                1784 non-null   int64  
 6   Title_x               1784 non-null   object 
 7   Body                  1784 non-null   object 
 8   CreatedAt             1784 non-null   object 
 9   ClosedAt              1732 non-null   object 
 10  MergedAt              1460 non-null   object 
 11  UpdatedAt             1784 non-null   object 
 12  State                 1784 non-null   object 
 13  Additions             1784 non-null   int64  
 14  Deletions             1784 non-null   int64  
 15  ChangedFiles         

In [29]:
# # Save to file.

# df_total.to_csv(os.path.join(dir_path, "cleaned", target + "_total.csv"))

## Merge Different Topics/JSON Files

In this project, we analyze data from `pull requests`, `issues`, and `discussions`.

To merge data from these different sources, we use the primary key `RepoName`, which is shared across all JSON files.

The following code reads data from previously cleaned CSV files, filters repositories using the programming language `Python` and the communication language `English`, selects relevant columns, and merges them into a consolidated dataset. Finally, the processed data is saved locally for future analysis.

In the end, we have 1321 rows and 4 columns in the merged dataset.

#### Read CSV Files

In [32]:
# Read CSV files.
df_pr = pd.read_csv("~/data/DevGPT/cleaned/pr_total.csv")
df_issue = pd.read_csv("~/data/DevGPT/cleaned/issue_total.csv")
df_discussion = pd.read_csv("~/data/DevGPT/cleaned/discussion_total.csv")

#### Remove Empty Answer and Prompt

In [33]:
# Clean them up
# Remove nan answer and promot.
df_pr = df_pr[~df_pr["Answer"].isna()]
df_pr = df_pr[~df_pr["Prompt"].isna()]

df_issue = df_issue[~df_issue["Answer"].isna()]
df_issue = df_issue[~df_issue["Prompt"].isna()]

df_discussion = df_discussion[~df_discussion["Answer"].isna()]
df_discussion = df_discussion[~df_discussion["Prompt"].isna()]

#### Filter out Python and English Prompt and Answer

In [34]:
# Filter out Python and English
df_pr = df_pr.loc[df_pr["RepoLanguage"] == "Python"]
df_issue = df_issue.loc[df_issue["RepoLanguage"] == "Python"]
df_discussion = df_discussion.loc[df_discussion["RepoLanguage"] == "Python"]

# Detect answer language
df_pr["Language"] = df_pr["Answer"].apply(detect)
df_issue["Language"] = df_issue["Answer"].apply(detect)
df_discussion["Language"] = df_discussion["Answer"].apply(detect)

# Get English only.
df_pr = df_pr.loc[df_pr["Language"] == "en"]
df_issue = df_issue.loc[df_issue["Language"] == "en"]
df_discussion = df_discussion.loc[df_discussion["Language"] == "en"]

print(len(df_pr), len(df_issue), len(df_discussion))

478 287 62


#### Unnest Prompt/Answer and Concatenation.

In [35]:
# Unnest prompt and answer
df_pr2 = df_pr[
    [
        "RepoName", "URL_chatgptsharing", "Prompt", "Answer", "TokensOfPrompts", "TokensOfAnswers"
        ]
    ].drop_duplicates()
df_pr3 = df_pr[
    [
        "RepoName", "URL_chatgptsharing_x", "Prompt", "Answer", "TokensOfPrompts", "TokensOfAnswers"
        ]
    ].drop_duplicates()
df_pr3.rename(columns={"URL_chatgptsharing_x": "URL_chatgptsharing"}, inplace=True)

df_pr4 = df_pr[
    [
        "RepoName", "URL_chatgptsharing_y", "Prompt", "Answer", "TokensOfPrompts", "TokensOfAnswers"
        ]
    ].drop_duplicates()
df_pr4.rename(columns={"URL_chatgptsharing_y": "URL_chatgptsharing"}, inplace=True)

df_issue2 = df_issue[
    [
        "RepoName", "URL_chatgptsharing", "Prompt", "Answer", "TokensOfPrompts", "TokensOfAnswers"
        ]
    ].drop_duplicates()
df_issue3 = df_issue[
    [
        "RepoName", "URL_chatgptsharing_x", "Prompt", "Answer", "TokensOfPrompts", "TokensOfAnswers"
        ]
    ].drop_duplicates()
df_issue3.rename(columns={"URL_chatgptsharing_x": "URL_chatgptsharing"}, inplace=True)

df_issue4 = df_issue[
    [
        "RepoName", "URL_chatgptsharing_y", "Prompt", "Answer", "TokensOfPrompts", "TokensOfAnswers"
        ]
    ].drop_duplicates()
df_issue4.rename(columns={"URL_chatgptsharing_y": "URL_chatgptsharing"}, inplace=True)

df_discussion2 = df_discussion[
    [
        "RepoName", "URL_chatgptsharing", "Prompt", "Answer", "TokensOfPrompts", "TokensOfAnswers"
        ]
    ].drop_duplicates()
df_discussion3 = df_discussion[
    [
        "RepoName", "URL_chatgptsharing_x", "Prompt", "Answer", "TokensOfPrompts", "TokensOfAnswers"
        ]
    ].drop_duplicates()
df_discussion3.rename(columns={"URL_chatgptsharing_x": "URL_chatgptsharing"}, inplace=True)

df_discussion4 = df_discussion[
    [
        "RepoName", "URL_chatgptsharing_y", "Prompt", "Answer", "TokensOfPrompts", "TokensOfAnswers"
        ]
    ].drop_duplicates()
df_discussion4.rename(columns={"URL_chatgptsharing_y": "URL_chatgptsharing"}, inplace=True)


print(len(df_pr2), len(df_pr3), len(df_pr4))
print(len(df_issue2), len(df_issue3), len(df_issue4))
print(len(df_discussion2), len(df_discussion3), len(df_discussion4))

201 201 201
204 204 218
39 39 41


In [36]:
# Concatenate final dataset.
df = pd.concat(
    [
        df_pr2, df_pr3, df_pr4, df_issue2, df_issue3, df_issue4, df_discussion2,
        df_discussion3, df_discussion4
        ]
    )
df.drop_duplicates()

,RepoName,URL_chatgptsharing,Prompt,Answer,TokensOfPrompts,TokensOfAnswers
7870,paul-gauthier/aider,https://chat.openai.com/share/4555f0ea-1e7b-49...,How can I setup a github action to automatical...,"Sure, I can certainly help you with that. Here...",31.0,474.0
7872,paul-gauthier/aider,https://chat.openai.com/share/4555f0ea-1e7b-49...,How can I setup a github action to automatical...,"Sure, I can certainly help you with that. Here...",0.0,0.0
7898,chitalian/gptask,https://chat.openai.com/share/902cd378-3ebc-4e...,Give me some test commands for this\n\nimport ...,This Python script is a command-line tool that...,980.0,301.0
7906,bbelderbos/htmx-demo,https://chat.openai.com/share/c8c101fa-aaae-49...,how to get the first 20 rows from a django model?,"To get the first 20 rows from a Django model, ...",340.0,941.0
7907,bbelderbos/htmx-demo,https://chat.openai.com/share/c8c101fa-aaae-49...,"I have this view for infinite scroll, would be...",Certainly! You can refactor the code to use Dj...,340.0,941.0
...,...,...,...,...,...,...
311,uuid6/prototypes,https://chat.openai.com/share/97dbf284-129f-42...,And more?,Certainly! Here are a few more options for a s...,206.0,2636.0
312,uuid6/prototypes,https://chat.openai.com/share/97dbf284-129f-42...,And more?,Certainly! Here are a few more options for a s...,206.0,2636.0
313,uuid6/prototypes,https://chat.openai.com/share/97dbf284-129f-42...,More.,Certainly! Here are a few more options for a s...,206.0,2636.0
111,sugi-01096/72,https://chat.openai.com/share/e2c50f86-6c14-4f...,import streamlit as st\nimport json\n\n\ndef s...,The code you provided seems to be a simple bul...,785.0,3270.0


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1348 entries, 7870 to 313
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   RepoName            1348 non-null   object 
 1   URL_chatgptsharing  1348 non-null   object 
 2   Prompt              1348 non-null   object 
 3   Answer              1348 non-null   object 
 4   TokensOfPrompts     1348 non-null   float64
 5   TokensOfAnswers     1348 non-null   float64
dtypes: float64(2), object(4)
memory usage: 73.7+ KB


#### Save Merged Dataset to File

In [38]:
# # Save to file

# df.to_csv(os.path.join(dir_path, "cleaned", "combine.csv"))